In [190]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
from pathlib import Path


In [191]:
folder = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Trade Openness/Supplementary Trade STATS - AUSTRALIA (1914)")
for f in folder.iterdir():
    print(f.name)

MANIFEST.TXT
oceania_1826_1938_FTWTHD_201710_v01.pdf
oceania_1826_1938_FTWTHD_201710_v01.xlsx


DATA EXTRACTION MAKING THE VARIABLE ---> REAL GDP per CAPITA 

In [192]:
# In the Paper they use the MAddison Project Database, For Japan missing data (1882, 1884)
# they use the 1885 value of real GDP per capita and population figures for 1882–1884 
# to back-calculate real GDP. 
# Taking constant real GDP in 1885, we divide it by the annual population in 1882, 1883 and 1884 
# to estimate real GDP per capita for those three years. This method provides a reasonable proxy 
# in 2011 international dollars. At the end of the day, we end up with data for the full sample of 16 countries 
# from 1882 to 2020.

# DATA file the Maddison Project Database found in the GDP folder in the DATA folder.

path_maddison_project = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/GDP/GDP per CAPITA - Maddison Project .xlsx")

df_maddison_project = pd.read_excel(path_maddison_project, sheet_name='Full data')

df_OECD_GDP = df_maddison_project[df_maddison_project['country'].isin(Project_Countries)]
df_OECD_GDP = df_OECD_GDP[['country', 'year', 'gdppc', 'pop']]
df_OECD_GDP = df_OECD_GDP[(df_OECD_GDP['year'] >= 1882) & (df_OECD_GDP['year'] <= 2020)]
df_OECD_GDP.head()

# Sorting out the data for Japan for the years 1882, 1883, 1884
mask_japan = df_OECD_GDP['country'] == 'Japan'
jp = df_OECD_GDP[mask_japan]

rgdp_1885 = (jp.loc[jp['year'] == 1885, 'gdppc'].iloc[0]* jp.loc[jp['year'] == 1885, 'pop'].iloc[0])
years_to_fill = [1882, 1883, 1884]
mask_japan_early = mask_japan & df_OECD_GDP['year'].isin(years_to_fill)

df_OECD_GDP.loc[mask_japan_early, 'gdppc'] = (rgdp_1885 / df_OECD_GDP.loc[mask_japan_early, 'pop'])

df_OECD_GDP.head()



,country,year,gdppc,pop
606,Australia,1882,6476.0,2348.0
607,Australia,1883,7133.0,2447.0
608,Australia,1884,6861.0,2556.0
609,Australia,1885,7049.0,2650.0
610,Australia,1886,6900.0,2741.0


Labour Productivity 

In [193]:
Project_Countries = [
    'Australia','Canada','Denmark','Finland','France','Germany',
    'Italy','Japan','Netherlands','Norway','Portugal','Spain',
    'Sweden','Switzerland','United Kingdom','United States'
]

path_labour_productivity = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Labour Productivity/Long Term Labour Productivity.xlsx")
df_labour_productivity = pd.read_excel(path_labour_productivity, sheet_name='Labor Productivity')

df_labour_productivity = df_labour_productivity[
    ['Year', 'AUS', 'AUT', 'BEL', 'CAN', 'DNK', 'FIN', 'FRA', 'DEU',
     'GRC', 'IRL', 'ITA', 'JPN', 'NLD', 'NZL', 'NOR', 'PRT', 'ESP',
     'SWE', 'CHE', 'GBR', 'USA']
]

replace_name_dict = {
    'AUS': 'Australia',
    'AUT': 'Austria',
    'BEL': 'Belgium',
    'CAN': 'Canada',
    'DNK': 'Denmark',
    'FIN': 'Finland',
    'FRA': 'France',
    'DEU': 'Germany',
    'GRC': 'Greece',
    'IRL': 'Ireland',
    'ITA': 'Italy',
    'JPN': 'Japan',
    'NLD': 'Netherlands',
    'NZL': 'New Zealand',
    'NOR': 'Norway',
    'PRT': 'Portugal',
    'ESP': 'Spain',
    'SWE': 'Sweden',
    'CHE': 'Switzerland',
    'GBR': 'United Kingdom',
    'USA': 'United States'
}

df_labour_productivity = df_labour_productivity.rename(columns=replace_name_dict)

df_labour_productivity = df_labour_productivity[
    (df_labour_productivity['Year'] >= 1890) &
    (df_labour_productivity['Year'] <= 2020)
]

columns_keep_df_labour_productivity = ["Year"] + Project_Countries
df_labour_productivity = df_labour_productivity[columns_keep_df_labour_productivity]
df_labour_productivity.head()

,Year,Australia,Canada,Denmark,Finland,France,Germany,Italy,Japan,Netherlands,Norway,Portugal,Spain,Sweden,Switzerland,United Kingdom,United States
90,1890,11.275570,4.185372,3.550943,2.023169,2.792736,4.814139,2.018100,1.020471,5.493740,3.641319,1.925517,4.093740,2.689252,5.961463,5.974749,4.655917
91,1891,11.346384,3.856720,3.622624,2.013744,2.798662,4.832038,2.087370,0.970538,5.331186,3.699415,1.899728,4.170664,2.619799,5.536116,5.999465,4.703721
92,1892,9.399105,3.786976,3.705531,1.930385,2.841674,4.893941,2.119804,1.034053,5.364090,3.734071,1.892896,4.459632,2.826159,5.927149,5.962112,4.626473
93,1893,8.518302,3.718733,3.764063,1.999724,2.756078,5.231450,2.181473,1.031995,5.351097,3.797245,1.935931,4.282428,2.976881,6.074320,5.974519,5.022901
94,1894,8.448742,3.847008,3.809196,2.107358,2.992517,5.341485,2.184280,1.154029,5.415974,3.785117,1.918264,4.309023,2.973899,5.848441,6.268525,5.131884


Consumer Price Index 

In [194]:
# Missibg Data from Ireland --> paper claims full data set but data set missing 1882 to 1921, linear interpolation not possible since we dont have a stating points cannot
# draw a trend line. 
path_CPI = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/CPI")
df_CPI_Jorda = pd.read_excel(path_CPI / " Jordá et al 2017  - .xlsx", sheet_name='Sheet1')
df_CPI_Jorda = df_CPI_Jorda[['country', 'year', 'cpi']]
df_CPI_Jorda = df_CPI_Jorda[(df_CPI_Jorda['year'] >= 1882) & (df_CPI_Jorda['year'] <= 2020)]
df_CPI_Jorda = df_CPI_Jorda[df_CPI_Jorda['country'].isin(Project_Countries)]


Trade Openness --> Exports - Imports / GDP 

In [200]:
# Trade Openness index --> Exports - Imports / GDP, keeping in line with the paper methodology
#Trade data are primarily sourced from the Jord`a-Schularick-Taylor Macrohistory Database (Jord`a et al., 2017)
#For specific country-year gaps, we first supplement with imports and exports data from Federico and Tena-Junguito (2016)
#Aus- tralia (1914), Switzerland (1882–1884), and Germany (1914–1923)
# In cases where the Jord`a-Schularick-Taylor Macrohistory Database provides data on imports and exports but lacks nominal GDP figures, 
# we supplement with nominal GDP data from the Global Macro Database (Mu ̈ller et al., 2025)
# we estimate trade openness by linearly interpolating the ratio of trade to GDP.
# full sample of 16 countries from 1882 to 2020.

path_TradeOpenness_Jorda_data_set = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Trade Openness")
df_TradeOpenness_Jorda_data_set = pd.read_excel(path_CPI / " Jordá et al 2017  - .xlsx", sheet_name='Sheet1')
df_TradeOpenness_Jorda_data_set = df_TradeOpenness_Jorda_data_set[['country', 'year', 'exports', 'imports','gdp']]
df_TradeOpenness_Jorda_data_set = df_TradeOpenness_Jorda_data_set[df_TradeOpenness_Jorda_data_set["country"].isin(Project_Countries)]

df_TradeOpenness_GMD_nominal_GDP_path = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Trade Openness")
df_TradeOpenness_GMD_nominal_GDP = pd.read_excel(df_TradeOpenness_GMD_nominal_GDP_path / "GMD.xlsx", sheet_name='data_final')
df_TradeOpenness_GMD_nominal_GDP = df_TradeOpenness_GMD_nominal_GDP[['countryname', 'year', 'nGDP', 'exports', 'imports']]
df_TradeOpenness_GMD_nominal_GDP = df_TradeOpenness_GMD_nominal_GDP.rename(columns={'countryname': 'country'})
df_TradeOpenness_GMD_nominal_GDP = df_TradeOpenness_GMD_nominal_GDP[df_TradeOpenness_GMD_nominal_GDP["country"].isin(Project_Countries)]
df_TradeOpenness_Jorda_data_set["country"] = (df_TradeOpenness_Jorda_data_set["country"].replace({'UK': 'United Kingdom','USA': 'United States'}))

df_TradeOpenness = pd.merge(df_TradeOpenness_Jorda_data_set, df_TradeOpenness_GMD_nominal_GDP, on=['country', 'year'], how='left')

df_Spdf_Supplementary_STATS_Australia_1914_path = Path("/Users/lorenzouberti/Desktop/Panel-Data-Project-Financial-Econometrics/Data/Trade Openness/Supplementary Trade STATS - AUSTRALIA (1914)")
df_Supplementary_STATS_Australia_1914 = pd.read_excel( df_Spdf_Supplementary_STATS_Australia_1914_path / "oceania_1826_1938_FTWTHD_201710_v01.xlsx", sheet_name='Current prices, 1913 borders')





Inputting the Missing Values for The exports and Imports and also the Nominal GDP 

In [205]:
# Australia (1914)
df_TradeOpenness.loc[
    (df_TradeOpenness["country"] == "Australia") &
    (df_TradeOpenness["year"] == 1914),
    "imports"
] = 343.33

df_TradeOpenness.loc[
    (df_TradeOpenness["country"] == "Australia") &
    (df_TradeOpenness["year"] == 1914),
    "exports"
] = 317.36


# Switzerland (1882, 1883, 1884)
swiss_years = [1882, 1883, 1884]

mask_ch = (
    (df_TradeOpenness["country"] == "Switzerland") &
    (df_TradeOpenness["year"].isin(swiss_years))
)

df_TradeOpenness.loc[mask_ch, "imports"] = [78.39, 87.47, 85.46]
df_TradeOpenness.loc[mask_ch, "exports"] = [134.97, 136.12, 136.68]


# Germany (1914–1923)
germany_years = [1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922, 1923]

mask_de = (
    (df_TradeOpenness["country"] == "Germany") &
    (df_TradeOpenness["year"].isin(germany_years))
)

df_TradeOpenness.loc[mask_de, "imports"] = [
    2023.86, 1690.52, 2000.05, 1690.52, 1690.52, 1858.55, 2051.43, 2112.12, 2122.92, 1529.60, 
]
df_TradeOpenness.loc[mask_de, "exports"] = [ 1761.95, 738.11, 904.78, 833.35, 1119.07, 1393.73, 1711.00, 1047.51, 1839.12, 1599.10 ] 

missing_gdp = df_TradeOpenness[df_TradeOpenness["nGDP"].isna()]
print("Missing nominal GDP rows:")
print(missing_gdp[["country", "year"]].head())


Missing nominal GDP rows:
     country  year
1359   Japan  1870
1360   Japan  1871
1361   Japan  1872
1362   Japan  1873
1363   Japan  1874


In [210]:
# Calculating Trade Openness
df_TradeOpenness['trade_openness'] = (df_TradeOpenness['exports_x'] + df_TradeOpenness['imports_x']) / df_TradeOpenness['nGDP']
#interpolating the Missing data statisitcs 

def interpolate_openness_for_country_years(df, country, start_year, end_year):
    # Work only with this country's data
    mask_country = df["country"] == country
    df_country = df.loc[mask_country].sort_values("year").copy()

    # Interpolate openness over the full series for that country
    df_country["openness_interp"] = df_country["trade_openness"].interpolate(method="linear")

    # Restrict to the target gap years
    mask_gap_years = (df_country["year"] >= start_year) & (df_country["year"] <= end_year)

    # Write interpolated values back into main df
    df.loc[mask_country & mask_gap_years, "trade_openness"] = df_country.loc[mask_gap_years, "openness_interp"].values

    return df

# Germany: 1944–1947
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Germany", 1944, 1947)

# Japan: 1944–1945
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Japan", 1944, 1945)

# Netherlands: 1944–1945
df_TradeOpenness = interpolate_openness_for_country_years(df_TradeOpenness, "Netherlands", 1944, 1945)


print(
    df_TradeOpenness.loc[(df_TradeOpenness["country"].isin(["Germany", "Japan", "Netherlands"])) &
         (df_TradeOpenness["year"].between(1943, 1948)),
         ["country", "year", "trade_openness"]]
    .sort_values(["country", "year"])
    .head(30)
)

          country  year  trade_openness
526       Germany  1943    1.976490e-04
527       Germany  1944    1.803004e-04
528       Germany  1945    1.629518e-04
529       Germany  1946    1.456032e-04
530       Germany  1947    1.282545e-04
531       Germany  1948    1.109059e-04
1432        Japan  1943    9.289249e-08
1433        Japan  1944    6.620317e-08
1434        Japan  1945    3.951384e-08
1435        Japan  1946    1.282452e-08
1436        Japan  1947    2.231536e-08
1437        Japan  1948    4.139276e-08
1583  Netherlands  1943    3.256909e-01
1584  Netherlands  1944    4.050163e-01
1585  Netherlands  1945    4.843416e-01
1586  Netherlands  1946    5.636670e-01
1587  Netherlands  1947    9.273169e-01
1588  Netherlands  1948    1.187786e+00
